In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionPipelineConfig:
    test_independent_data_path: Path
    test_dependent_data_path: Path
    trained_model_path: Path
    predictions_output_path: Path

In [3]:
import os
os.chdir('../')

In [6]:
from src.No_More_Lapses.constants import *
from src.No_More_Lapses.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prediction_pipeline_config(self) -> PredictionPipelineConfig:
        config = self.config.prediction_pipeline

        
        data_transformation_config = PredictionPipelineConfig(
            test_independent_data_path = config.test_independent_data_path,
            test_dependent_data_path = config.test_dependent_data_path,
            trained_model_path = config.trained_model_path,
            predictions_output_path=config.predictions_output_path
        )

        return data_transformation_config

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import mlflow
import os
from sklearn.preprocessing import StandardScaler
from src.No_More_Lapses import logger
from src.No_More_Lapses.components.attention_model import AttentionLayer
from src.No_More_Lapses.entity.config_entity import PredictionPipelineConfig

class PredictionPipeline:
    def __init__(self, config: PredictionPipelineConfig):
        self.config = config

    def run_prediction(self):
        # Load test data
        X_test = pd.read_csv(self.config.test_independent_data_path)
        y_test = pd.read_csv(self.config.test_dependent_data_path)

        # Drop index column if present
        X_test = X_test.drop(columns=['Unnamed: 0'], errors='ignore')
        y_test = y_test.drop(columns=['Unnamed: 0'], errors='ignore')

        logger.info("Loaded and cleaned test data.")

        # Scale test data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_test)  # ⚠️ If you saved the training scaler, load and use it here instead

        # Load trained model

        model = tf.keras.models.load_model(
                self.config.trained_model_path,
                compile=False,
                custom_objects={"AttentionLayer": AttentionLayer}
            )

        logger.info("Trained model loaded.")

        # Predict probabilities
        y_pred_probs = model.predict(X_scaled)
        y_pred_labels = np.argmax(y_pred_probs, axis=1)

        # Add lapse_calculator column to test DataFrame
        results = X_test.copy()
        results["lapse_calculator"] = y_pred_labels

        # Save result
        os.makedirs(os.path.dirname(self.config.predictions_output_path), exist_ok=True)
        results.to_csv(self.config.predictions_output_path, index=False)
        logger.info(f"Predictions saved to: {self.config.predictions_output_path}")

        # Log as MLflow artifact
        with mlflow.start_run(run_name="attentionModel_prediction"):
            mlflow.log_artifact(self.config.predictions_output_path)


In [14]:
config = ConfigurationManager().get_prediction_pipeline_config()
prediction_stage = PredictionPipeline(config=config)
prediction_stage.run_prediction()

[2025-03-24 19:47:12,984: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-24 19:47:12,987: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-24 19:47:12,988: INFO: common: created directory at: artifacts]
[2025-03-24 19:47:13,030: INFO: 3581776736: Loaded and cleaned test data.]
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2025-03-24 19:47:13.064660: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-24 19:47:13.065295: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


[2025-03-24 19:47:13,379: INFO: 3581776736: Trained model loaded.]
   1/1160 [..............................] - ETA: 3:39

2025-03-24 19:47:13.422752: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-03-24 19:47:13.500176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1160/1160 [==============================] - 3s 2ms/step
[2025-03-24 19:47:16,825: INFO: 3581776736: Predictions saved to: artifacts/predictions/lapse_predictions.csv]
